In [ ]:
# import sys
# !{sys.executable} -m pip install imbalanced-learn --user
# !{sys.executable} -m pip install hyperopt --user

In [1]:
import numpy
from time import time
from sklearn.model_selection import GridSearchCV
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split
from copy import deepcopy

from newsgac import config
from newsgac.genres import genre_codes
from newsgac.learners import learners, LearnerSVC, LearnerNB, LearnerXGB, LearnerGB, LearnerMLP, LearnerRF, LearnerLGBM
from newsgac.pipelines.get_sk_pipeline import get_sk_pipeline
from newsgac.pipelines.utils import report
from sklearn.ensemble import RandomForestClassifier


from newsgac import database
from newsgac.data_sources import DataSource
from newsgac.pipelines import Pipeline

# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
# from imblearn.combine import SMOTEENN, SMOTETomek
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/newsgac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
[d.display_title for d in DataSource.objects.all()]

[u'N2BGS Training',
 u'N2BGS Testing',
 u'Linked NRC (2930)',
 u'BOW + 5 features (N2BGS train)',
 u'BOW + 5 features (N2BGS test)',
 u'nrc-1950-1994-a.txt',
 u'nrc-1950-1994-b.txt',
 u'nrc-1950-1994-c.txt',
 u'nrc-1950-1994-d.txt',
 u'nrc-1950-1994-e.txt',
 u'telegraaf-1950-1994-a.txt',
 u'telegraaf-1950-1994-b.txt',
 u'telegraaf-1950-1994-c.txt',
 u'telegraaf-1950-1994-d.txt',
 u'telegraaf-1950-1994-e.txt',
 u'volkskrant-1950-1995-a.txt',
 u'volkskrant-1950-1995-b.txt',
 u'volkskrant-1950-1995-c.txt',
 u'volkskrant-1950-1995-d.txt',
 u'volkskrant-1950-1995-e.txt',
 u'nrc-1965.txt',
 u'nrc-1985.txt',
 u'BOW + 9 features (N2BGS train)',
 u'BOW + 9 features (N2BGS test)',
 u'Linked NRC (2930/9 features)',
 u'BOW Train unique (9 features)',
 u'BOW Test unique (9 features)',
 u'Linked NRC (unique/9 features)',
 u'BOW Train unique (N3BGS/9 features)',
 u'N3BGS FROG Test',
 u'N3BGS FROG Train',
 u'BOW Test unique (N3BGS/9 features)',
 u'BOW Test unique (N3BGS/9 features/collapsed)',
 u'Link

In [3]:
[d.display_title for d in Pipeline.objects.all()]

[u'dummy frog',
 u'dummy bow',
 u'RF FROG',
 u'SVC FROG',
 u'XGB FROG',
 u'NB FROG',
 u'MLP FROG',
 u'RF BOW',
 u'SVC BOW',
 u'XGB BOW',
 u'NB BOW',
 u'MLP BOW',
 u'SVC LIN BOW',
 u'Erik MLP BOW (with stop-words)',
 u'Erik MLP BOW (5 features) ',
 u'Erik MLP BOW (9 features) ',
 u'Erik SVC BOW (9 features)',
 u'Erik RF BOW (9 features)',
 u'Erik XGB BOW (9 features)',
 u'Erik NB BOW (9 features)',
 u'Erik MLP unique 9 features',
 u'Erik NB unique (9 features)',
 u'Erik XGB unique (9 features)',
 u'Erik SVC unique (9 features)',
 u'Erik RF unique (9 features)',
 u'N3BGS MLP unique 9 features',
 u'N3BGS NB unique 9 features ',
 u'N3BGS SVC unique 9 features ',
 u'N3BGS RF unique 9 features',
 u'N3BGS XGB unique 9 features ',
 u'dummy n3bgs bow',
 u'dummy n3bgs frog',
 u'RF FROG N3BGS',
 u'SVC FROG N3BGS',
 u'XGB FROG N3BGS',
 u'Erik NB N3BGS Collapsed',
 u'Erik MLP N3BGS Collapsed',
 u'Erik MLP Linked NRC (2930 articles)',
 u'Erik RF Linked NRC (2930 articles) ',
 u'Erik NB Linked NRC (2

In [4]:
p = Pipeline.objects.all()[65]
print 'DataSource: ' + p.data_source.display_title
print 'NLP Tool: ' + p.nlp_tool.name
print 'Classifier: ' + p.learner.name
print 'Task status: ' + str(p.task.status)

DataSource: 20190228 unbalanced train size=3443
NLP Tool: TF-IDF
Classifier: Light GBM
Task status: Status.SUCCESS


In [5]:
len(p.data_source.articles)

3099

In [6]:
test_data_source = DataSource.objects.all()[39]
print 'Testing data source: ' + test_data_source.display_title

Testing data source: 20190228 unbalanced test size=3443


In [7]:
skp = p.get_sk_pipeline()
skp_opt = deepcopy(skp)
skp_opt.steps.pop()
skp_opt.steps

[('CleanOCR', <newsgac.nlp_tools.transformers.CleanOCR at 0x7f3a9079f890>),
 ('StopWordRemoval',
  <newsgac.nlp_tools.transformers.StopWordRemoval at 0x7f3a9079f8d0>),
 ('FeatureExtraction', FeatureUnion(n_jobs=None,
         transformer_list=[('TFIDF', Pipeline(memory=None,
       steps=[('RemoveQuotes', <newsgac.nlp_tools.transformers.RemoveQuotes object at 0x7f3a9079f9d0>), ('TF-IDF', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
          dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
          lowercase=True, max_df=1.0, max_featu...        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('RobustScaler',
  RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=False,
         with_scaling=True))]

In [8]:
texts = numpy.array([article.raw_text for article in p.data_source.articles])
labels = numpy.array([article.label for article in p.data_source.articles])

X = skp_opt.fit_transform(texts)

In [9]:
# get the test dataset for optimization accuracy
texts_test = numpy.array([article.raw_text for article in test_data_source.articles])
labels_test = numpy.array([article.label for article in test_data_source.articles])

X_test = skp_opt.transform(texts_test)

In [10]:
X_train = X
y_train = labels
y_test = labels_test

In [11]:
y_test.shape

(344,)

In [12]:
criterion_list = ['gini', 'entropy']
n_estimators_list = np.arange(1, 1000, 1, dtype=int)
max_features_list = np.arange(1, X_train.shape[1], 1, dtype=int) 
max_depth_list = np.arange(1, 50, 1, dtype=int) 
min_samples_split_list = np.arange(10, 100, 10, dtype=int)
min_samples_leaf_list = np.arange(1, 30, 1, dtype=int)
max_leaf_nodes_list = np.arange(10, 30, 2, dtype=int)
class_weights = [None, 'balanced']
bootstrap_list =[True, False]


In [13]:
def optimize(evals, trials, optimizer=tpe.suggest):
    space = {
        'criterion': hp.choice('criterion', criterion_list),
        'n_estimators': hp.choice('n_estimators', n_estimators_list),
        'max_features': hp.choice('max_features', max_features_list),
        'max_depth': hp.choice('max_depth', max_depth_list),
        'min_samples_split': hp.choice('min_samples_split', min_samples_split_list),
        'min_samples_leaf': hp.choice('min_samples_leaf', min_samples_leaf_list),
        'max_leaf_nodes': hp.choice('max_leaf_nodes', max_leaf_nodes_list),
        'class_weight': hp.choice('class_weight', class_weights),
        'bootstrap': hp.choice('bootstrap', bootstrap_list)
    }
    best = fmin(score, space, algo=optimizer, max_evals=evals, trials=trials)
    pbar.close()
    return best

In [14]:
# def train_test_split_w_sampling(X, y, sampling_method, test_size):
#     if sampling_method == 'under-sampling':
#         random_under_sampler = RandomUnderSampler(ratio='majority', random_state=0)
#         X, y = random_under_sampler.fit_sample(X, y)

#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

#         return X_train, X_test, y_train, y_test
#     elif sampling_method == 'over-sampling':
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

#         random_over_sampler = RandomOverSampler(ratio='minority', random_state=0)
#         X_train, y_train = random_over_sampler.fit_sample(X_train, y_train)

#         return X_train, X_test, y_train, y_test
#     elif sampling_method == 'smote':
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

#         smote = SMOTE(random_state=0, n_jobs=6)
#         X_train, y_train = smote.fit_sample(X_train, y_train)
        
#         return X_train, X_test, y_train, y_test
#     elif sampling_method == 'smote_enn':
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

#         smote_enn = SMOTEENN(random_state=0)
#         X_train, y_train = smote_enn.fit_sample(X_train, y_train)
        
#         return X_train, X_test, y_train, y_test
#     elif sampling_method == 'smote_tomek':
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)

#         smote_tomek = SMOTETomek(random_state=0)
#         X_train, y_train = smote_tomek.fit_sample(X_train, y_train)
        
#         return X_train, X_test, y_train, y_test
#     else:
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
        
#         return X_train, X_test, y_train, y_test

In [15]:
# X_train, X_test, y_train, y_test = train_test_split_w_sampling(X, labels, 'no sampling', 0.1)
# X_train, X_test, y_train, y_test = train_test_split_w_sampling(X, labels, 'under-sampling', 0.33)

In [16]:
train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []

def score(params):
    criterion = params['criterion']
    n_estimators = params['n_estimators']
    max_features = params['max_features']
    max_depth = params['max_depth']
    min_samples_split = params['min_samples_split']
    min_samples_leaf = params['min_samples_leaf']
    max_leaf_nodes = params['max_leaf_nodes']
    class_weight = params['class_weight']
    bootstrap = params['bootstrap']
    
    model = RandomForestClassifier(criterion=criterion, n_estimators=n_estimators, max_features=max_features,
                                  max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
                                  max_leaf_nodes=max_leaf_nodes, class_weight=class_weight, bootstrap=bootstrap,
                                  random_state=42, n_jobs=48)
#     model = RandomForestClassifier(criterion=criterion)
    
    model.fit(X_train, y_train)
    labels_prediction = model.predict(X_test)
    labels_probs = model.predict_proba(X_test)
    train_acc = accuracy_score(y_train, model.predict(X_train))
    train_loss = log_loss(y_train, model.predict_proba(X_train))
    test_acc = accuracy_score(y_test, labels_prediction)
    test_loss = log_loss(y_test, labels_probs)
    
    train_acc_list.append(train_acc)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_acc)
    test_loss_list.append(test_loss)
    
    print('Parameters with this training accuracy {} and loss {} :'.format(train_acc, train_loss))
    print('Parameters with this testing accuracy {} and loss {} :'.format(test_acc, test_loss))
    print(params)
    pbar.update()
#     return {'loss': test_loss, 'status': STATUS_OK}
    return {'loss': -test_acc, 'status': STATUS_OK}

In [ ]:
import time
from tqdm import tqdm
import numpy as np

trials = Trials()
cores = 48
start = time.time()
evaluations = 1000
pbar = tqdm(total=evaluations, desc="Hyperopt")
best_param = optimize(evals=evaluations,
                      optimizer=tpe.suggest,
                      trials=trials)
print("------------------------------------")
print("The best hyperparameters are: ", "\n")
print(best_param)
end = time.time()
print('Time elapsed to optimize {0} executions: {1}'.format(evaluations, end - start))
best_param['criterion'] = criterion_list[best_param['criterion']]
best_param['max_leaf_nodes'] = max_leaf_nodes_list[best_param['max_leaf_nodes']]
best_param['min_samples_leaf'] = min_samples_leaf_list[best_param['min_samples_leaf']]
best_param['n_estimators'] = n_estimators_list[best_param['n_estimators']]
best_param['max_features'] = max_features_list[best_param['max_features']]
best_param['min_samples_split'] = min_samples_split_list[best_param['min_samples_split']]
best_param['max_depth'] = max_depth_list[best_param['max_depth']]
best_param['class_weight'] = class_weights[best_param['class_weight']]
best_param['bootstrap'] = bootstrap_list[best_param['bootstrap']]
print('\n Best score:')
score(best_param)

Hyperopt:   0%|          | 0/1000 [00:00<?, ?it/s]


  0%|          | 0/1000 [00:00<?, ?it/s, best loss: ?]
Parameters with this training accuracy 0.257502420136 and loss 2.06310038833 :

  0%|          | 0/1000 [00:14<?, ?it/s, best loss: ?]
Parameters with this testing accuracy 0.191860465116 and loss 2.17972585961 :

  0%|          | 0/1000 [00:14<?, ?it/s, best loss: ?]
{'max_leaf_nodes': 18, 'bootstrap': True, 'min_samples_leaf': 22, 'n_estimators': 725, 'max_features': 14210, 'criterion': 'entropy', 'min_samples_split': 30, 'max_depth': 6, 'class_weight': 'balanced'}

  0%|          | 0/1000 [00:14<?, ?it/s, best loss: ?]

Hyperopt:   0%|          | 1/1000 [00:24<6:53:59, 24.86s/it]


  0%|          | 1/1000 [00:14<3:59:33, 14.39s/it, best loss: -0.191860465116]
Parameters with this training accuracy 0.212326556954 and loss 2.12557894738 :

  0%|          | 1/1000 [00:47<3:59:33, 14.39s/it, best loss: -0.191860465116]
Parameters with this testing accuracy 0.156976744186 and loss 2.38662736599 :

  0%|          | 1/1000 [00:47<3:59:33, 14.39s/it, best loss: -0.191860465116]
{'max_leaf_nodes': 22, 'bootstrap': False, 'min_samples_leaf': 28, 'n_estimators': 984, 'max_features': 26116, 'criterion': 'entropy', 'min_samples_split': 70, 'max_depth': 48, 'class_weight': 'balanced'}

  0%|          | 1/1000 [00:47<3:59:33, 14.39s/it, best loss: -0.191860465116]

Hyperopt:   0%|          | 2/1000 [00:57<7:32:33, 27.21s/it]

...


 13%|█▎        | 128/1000 [24:34<2:29:06, 10.26s/it, best loss: -0.56976744186]
Parameters with this training accuracy 0.541142303969 and loss 1.48343244376 :

 13%|█▎        | 128/1000 [24:45<2:29:06, 10.26s/it, best loss: -0.56976744186]
Parameters with this testing accuracy 0.540697674419 and loss 1.50081347364 :

 13%|█▎        | 128/1000 [24:45<2:29:06, 10.26s/it, best loss: -0.56976744186]
{'max_leaf_nodes': 12, 'bootstrap': True, 'min_samples_leaf': 19, 'n_estimators': 625, 'max_features': 17985, 'criterion': 'gini', 'min_samples_split': 70, 'max_depth': 20, 'class_weight': None}

 13%|█▎        | 128/1000 [24:45<2:29:06, 10.26s/it, best loss: -0.56976744186]

Hyperopt:  13%|█▎        | 129/1000 [24:55<2:33:22, 10.57s/it]


 13%|█▎        | 129/1000 [24:45<2:33:22, 10.57s/it, best loss: -0.56976744186]
Parameters with this training accuracy 0.585672797677 and loss 1.33131933877 :

 13%|█▎        | 129/1000 [24:54<2:33:22, 10.57s/it, best loss: -0.56976744186]
Parameters with this testing accuracy 0.563953488372 and loss 1.4222417147 :

 13%|█▎        | 129/1000 [24:54<2:33:22, 10.57s/it, best loss: -0.56976744186]
{'max_leaf_nodes': 24, 'bootstrap': False, 'min_samples_leaf': 9, 'n_estimators': 324, 'max_features': 14071, 'criterion': 'gini', 'min_samples_split': 30, 'max_depth': 26, 'class_weight': None}

 13%|█▎        | 129/1000 [24:54<2:33:22, 10.57s/it, best loss: -0.56976744186]

Hyperopt:  13%|█▎        | 130/1000 [25:04<2:25:48, 10.06s/it]


 13%|█▎        | 130/1000 [24:54<2:25:47, 10.06s/it, best loss: -0.56976744186]
Parameters with this training accuracy 0.327847692804 and loss 1.93778364818 :

 13%|█▎        | 130/1000 [25:00<2:25:47, 10.06s/it, best loss: -0.56976744186]


In [ ]:
# get the best params and train the model
RF_opt = RandomForestClassifier(criterion=best_param['criterion'], n_estimators=best_param['n_estimators'], max_features=best_param['max_features'],
                                  max_depth=best_param['max_depth'], min_samples_split=best_param['min_samples_split'], min_samples_leaf=best_param['min_samples_leaf'],
                                  max_leaf_nodes=best_param['max_leaf_nodes'], class_weight=best_param['class_weight'], bootstrap=best_param['bootstrap'],
                                  random_state=42, n_jobs=48)


fitted_model = RF_opt.fit(X=X_train,y=y_train)

# train_acc = accuracy_score(y_train, fitted_model.predict(X_train))
# train_loss = log_loss(y_train, fitted_model.predict_proba(X_train))
# test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
# test_loss = log_loss(y_test, fitted_model.predict_proba(X_test))

# print('Parameters with this training accuracy {} and loss {} :'.format(train_acc, train_loss))
# print('Parameters with this testing accuracy {} and loss {} :'.format(test_acc, test_loss))


In [ ]:
y_score = fitted_model.predict_proba(X_test)
y_score

In [ ]:
test_acc = accuracy_score(y_test, fitted_model.predict(X_test))
test_acc

In [ ]:
from matplotlib import pyplot as plt 
%matplotlib inline 
x_list =range(len(train_acc_list))

plt.plot(x_list, train_acc_list, label='Training accuracy')
plt.plot(x_list, test_acc_list, label='Testing accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(x_list, train_loss_list, label='Training loss')
plt.plot(x_list, test_loss_list, label='Testing loss')
plt.legend()
plt.show()